In [70]:
import pandas as pd
import numpy as np

In [31]:
spotify_cities = pd.read_csv('cities.csv')
countries = pd.read_csv('countries.csv')
world_pop = pd.read_csv('worldcitiespop.csv', sep=';')
iso3166 = pd.read_csv('iso3166-2.csv', header=None, names=['code', 'human_region'])
world_cities = pd.read_csv('worldcities.csv')
world_cities2 = pd.read_csv('World_Cities.csv')
geonames = pd.read_csv('geonames-all-cities-with-a-population-1000.csv', sep=';')

In [3]:
# Removing HK and SG for now
spotify_cities = spotify_cities[~spotify_cities.country.isin(['HK', 'SG'])].drop_duplicates()

In [4]:
spotify_cities['iso3166'] = spotify_cities['country'] + '-' + spotify_cities['region']

In [5]:
spotify_cities = spotify_cities.merge(countries, left_on='country', right_on='Code', how='left')
spotify_cities = spotify_cities.merge(iso3166, left_on='iso3166', right_on='code', how='left')

In [6]:
spotify_cities = spotify_cities[['city', 'country', 'region', 'human_region', 'Name']]
spotify_cities = spotify_cities.rename(columns={"Name": "human_country"})
spotify_cities.head()

,city,country,region,human_region,human_country
0,South Yarra,AU,VIC,Victoria,Australia
1,Grand Prairie,US,TX,Texas,United States
2,Kongens Lyngby,DK,84,Hovedstaden,Denmark
3,Blackburn,GB,BBD,Blackburn with Darwen,United Kingdom
4,Walsall,GB,WLL,Walsall,United Kingdom


In [7]:
spotify = spotify_cities.merge(world_cities, left_on=['country', 'human_region', 'city'], right_on=['iso2', 'admin_name', 'city'], how='left')

In [8]:
spotify = spotify[['city', 'country_x', 'region', 'human_region', 'human_country', 'population', 'lat', 'lng']]
spotify = spotify.rename(columns={"country_x": "country"})

In [9]:
spotify.head()

,city,country,region,human_region,human_country,population,lat,lng
0,South Yarra,AU,VIC,Victoria,Australia,NaN,NaN,NaN
1,Grand Prairie,US,TX,Texas,United States,193837.0,32.687,-97.0209
2,Kongens Lyngby,DK,84,Hovedstaden,Denmark,NaN,NaN,NaN
3,Blackburn,GB,BBD,Blackburn with Darwen,United Kingdom,NaN,53.750,-2.4833
4,Walsall,GB,WLL,Walsall,United Kingdom,NaN,52.600,-2.0000


In [10]:
# Matched — coordinates and population found
unmatched = spotify[spotify.lat.isnull()]
matched = spotify[~spotify.lat.isnull()]

In [11]:
print(unmatched.shape)
print(matched.shape)
unmatched = unmatched[['city', 'country', 'region', 'human_region', 'human_country']]

(2684, 8)
(1540, 8)


In [12]:
world_pop = world_pop[['Country', 'AccentCity', 'Population', 'Latitude', 'Longitude']]
world_pop['ctr'] = world_pop['Country'].str.upper()
world_pop.head()

,Country,AccentCity,Population,Latitude,Longitude,ctr
0,tr,Tepeagzi,NaN,40.920404,39.192090,TR
1,tr,Yucepinar,NaN,40.246859,35.514880,TR
2,ua,Zmiyivka,NaN,46.867656,33.583913,UA
3,ua,Belka,NaN,50.523429,29.461285,UA
4,us,Lost Creek,NaN,37.481667,-83.325278,US


In [13]:
with_duplicates = unmatched.merge(world_pop, left_on=['country', 'city'], right_on=['ctr', 'AccentCity'], how='left')
with_duplicates.head()

,city,country,region,human_region,human_country,Country,AccentCity,Population,Latitude,Longitude,ctr
0,South Yarra,AU,VIC,Victoria,Australia,au,South Yarra,NaN,-37.833333,144.983333,AU
1,Kongens Lyngby,DK,84,Hovedstaden,Denmark,dk,Kongens Lyngby,NaN,55.771785,12.505991,DK
2,Lebbeke,BE,VOV,Oost-Vlaanderen,Belgium,be,Lebbeke,17372.0,51.000000,4.133333,BE
3,Zaventem,BE,VBR,Vlaams-Brabant,Belgium,be,Zaventem,28188.0,50.883333,4.466667,BE
4,Wetteren,BE,VOV,Oost-Vlaanderen,Belgium,be,Wetteren,22930.0,51.000000,3.883333,BE


In [14]:
with_duplicates = with_duplicates[['city', 'country', 'region', 'human_region', 'human_country', 'Population', 'Latitude', 'Longitude']]
with_duplicates = with_duplicates.rename(columns={"Population": "population", "Latitude": "lat", "Longitude": "lng"})

In [15]:
with_duplicates.head()

,city,country,region,human_region,human_country,population,lat,lng
0,South Yarra,AU,VIC,Victoria,Australia,NaN,-37.833333,144.983333
1,Kongens Lyngby,DK,84,Hovedstaden,Denmark,NaN,55.771785,12.505991
2,Lebbeke,BE,VOV,Oost-Vlaanderen,Belgium,17372.0,51.000000,4.133333
3,Zaventem,BE,VBR,Vlaams-Brabant,Belgium,28188.0,50.883333,4.466667
4,Wetteren,BE,VOV,Oost-Vlaanderen,Belgium,22930.0,51.000000,3.883333


In [16]:
uniqs = with_duplicates[~with_duplicates.duplicated(subset=['country', 'region', 'city'], keep=False)]

In [17]:
uniqs_matched = uniqs[~uniqs.lat.isnull()]

In [18]:
uniqs_matched.shape

(1609, 8)

In [19]:
dups = with_duplicates[with_duplicates.duplicated(subset=['country', 'region', 'city'], keep=False)]

In [20]:
unduped = dups.sort_values(['city', 'country', 'region', 'population'])

In [21]:
unduped = unduped.drop_duplicates(subset=['country', 'region', 'city'])

In [23]:
mega = pd.concat([matched, uniqs_matched, unduped])

In [39]:
no_pop = mega[mega.population.isnull()]
no_pop.count()

city             797
country          797
region           794
human_region     794
human_country    797
population         0
lat              797
lng              797
dtype: int64

In [40]:
yes_pop = no_pop.merge(geonames, left_on=['city', 'country'], right_on=['Name', 'Country Code'])
yes_pop = yes_pop[['city', 'country', 'region', 'human_region', 'human_country', 'Population', 'lat', 'lng']]
yes_pop = yes_pop.rename(columns={"Population": "population"})

In [51]:
yes_pop_sorted = yes_pop.sort_values(['city', 'country', 'region', 'population'], ascending=False)
yes_pop_unduped = yes_pop_sorted.drop_duplicates(subset=['country', 'region', 'city'])
yes_pop_unduped.shape

(463, 8)

In [52]:
mega_pop = mega[~mega.population.isnull()]
mega_pop.shape

(2851, 8)

In [53]:
final = pd.concat([mega_pop, yes_pop_unduped], ignore_index=True)

In [57]:
final = final.append({'city': 'Hong Kong', 'country': 'HK', 'region': '', 'human_region': '', 'human_country': 'Hong Kong', 'population': 7392000, 'lat':22.321542, 'lng':114.173011}, ignore_index=True, sort=False)
final = final.append({'city': 'Singapore', 'country': 'SG', 'region': '', 'human_region': '', 'human_country': 'Singapore', 'population': 5612000, 'lat':1.295121, 'lng':103.836695}, ignore_index=True, sort=False)

In [60]:
final = final.astype({'population': 'int32'})

In [71]:
final = final[final.population != 0]
final.index = np.arange(1, len(final) + 1)

In [77]:
final['coord'] = final.apply(lambda r: "SRID=4326;Point(" + str(r.lng) + " " + str(r.lat) + ")", axis=1)

In [78]:
final.head()

,city,country,region,human_region,human_country,population,lat,lng,coord
1,Grand Prairie,US,TX,Texas,United States,193837,32.6870,-97.0209,SRID=4326;Point(-97.0209 32.687)
2,Columbus,US,GA,Georgia,United States,259160,32.5100,-84.8771,SRID=4326;Point(-84.8771 32.51)
3,Tarija,BO,T,Tarija,"Bolivia, Plurinational State of",159269,-21.5167,-64.7500,SRID=4326;Point(-64.75 -21.5167)
4,Sucre,BO,H,Chuquisaca,"Bolivia, Plurinational State of",224838,-19.0410,-65.2595,SRID=4326;Point(-65.2595 -19.041)
5,Oruro,BO,O,Oruro,"Bolivia, Plurinational State of",246501,-17.9800,-67.1300,SRID=4326;Point(-67.13 -17.98)


In [80]:
final[['city', 'region', 'country', 'human_region', 'human_country', 'population', 'coord']].to_csv('final_cities.csv')